<a href="https://colab.research.google.com/github/Diego87Aoki/aula11_tradu-o_de_TEXTO/blob/main/Atividade_IA_em_Nuvem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# ETAPA 2 (Tradutor híbrido) - Cole e execute no Google Colab
# Autor: Assistente (engenheiro de software)
# Instala dependências (execute apenas uma vez)
!pip install -q transformers sentencepiece requests pillow

# -------- IMPORTS --------
import requests
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from typing import Tuple

# -------- CONFIG --------
# Servidores LibreTranslate (tenta em ordem)
LIBRE_SERVERS = [
    "https://translate.astian.org/translate",
    "https://libretranslate.de/translate",
    "https://libretranslate.coldlar.me/translate",
    "https://translate.argosopentech.com/translate"
]

# Pares OPUS-MT que vamos suportar no fallback offline
# Você pode ampliar essa lista adicionando mais modelos do Helsinki-NLP
OPUS_MODELS = {
    ("pt", "en"): "Helsinki-NLP/opus-mt-pt-en",
    ("en", "pt"): "Helsinki-NLP/opus-mt-en-pt",
    ("pt", "es"): "Helsinki-NLP/opus-mt-pt-es",
    ("es", "pt"): "Helsinki-NLP/opus-mt-es-pt",
    ("en", "es"): "Helsinki-NLP/opus-mt-en-es",
    ("es", "en"): "Helsinki-NLP/opus-mt-es-en",
    ("en", "fr"): "Helsinki-NLP/opus-mt-en-fr",
    ("fr", "en"): "Helsinki-NLP/opus-mt-fr-en"
}

# Dispositivo (usa GPU se disponível)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando device: {DEVICE}")

# -------- FUNÇÃO: Tentar API LibreTranslate --------
def translate_via_libre(text: str, src: str, tgt: str, timeout: int = 6) -> Tuple[str, str]:
    """
    Tenta vários endpoints LibreTranslate. Retorna (translated_text, engine) ou (None, None) se falhar.
    src/tgt são códigos ISO de 2 letras (ex: 'pt', 'en').
    """
    payload = {"q": text, "source": src, "target": tgt, "format": "text"}
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    for url in LIBRE_SERVERS:
        try:
            resp = requests.post(url, data=payload, headers=headers, timeout=timeout)
            if resp.status_code == 200:
                j = resp.json()
                # resposta esperada: {"translatedText": "..." }
                if isinstance(j, dict) and "translatedText" in j:
                    return j["translatedText"], f"libre:{url}"
                # heurística para respostas diferentes
                if isinstance(j, dict):
                    for v in j.values():
                        if isinstance(v, str) and v.strip():
                            return v, f"libre:{url}"
        except Exception:
            # ignora e tenta o próximo servidor
            continue
    return None, None

# -------- FUNÇÃO: Fallback OFFLINE (OPUS-MT) --------
# Cache de modelos/tokenizers carregados
_loaded_tokenizers = {}
_loaded_models = {}

def _load_opus_model(model_name: str):
    """
    Carrega tokenizer e modelo OPUS-MT (caching).
    Move o modelo para DEVICE.
    """
    if model_name in _loaded_tokenizers:
        return _loaded_tokenizers[model_name], _loaded_models[model_name]
    print(f"🔄 Baixando e carregando modelo OPUS-MT: {model_name} (pode demorar alguns segundos)...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    model.to(DEVICE)
    _loaded_tokenizers[model_name] = tokenizer
    _loaded_models[model_name] = model
    print("✅ Modelo carregado:", model_name)
    return tokenizer, model

def translate_offline_opus(text: str, src: str, tgt: str) -> Tuple[str, str]:
    """
    Faz tradução usando modelos OPUS-MT locais para pares suportados.
    Retorna (translated_text, engine) ou (None, reason) se não suportado.
    """
    key = (src, tgt)
    if key not in OPUS_MODELS:
        return None, "offline:par_nao_suportado"
    model_name = OPUS_MODELS[key]
    try:
        tokenizer, model = _load_opus_model(model_name)
        # Tokenizar e gerar
        inputs = tokenizer(text, return_tensors="pt", padding=True).to(DEVICE)
        outputs = model.generate(**inputs, max_length=256)
        translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return translated, f"offline:{model_name}"
    except Exception as e:
        return None, f"offline:error:{e}"

# -------- FUNÇÃO HÍBRIDA PRINCIPAL --------
def traduzir(text: str, src: str, tgt: str) -> Tuple[str, str]:
    """
    Estratégia:
      1) tenta LibreTranslate (múltiplos servidores)
      2) se falhar, tenta OPUS-MT local para pares suportados
      3) se ainda falhar, retorna o texto original com razão
    Retorna: (translated_text, engine_used)
    """
    # Sanity: lower-case codes
    src = src.lower()
    tgt = tgt.lower()

    # 1) API
    api_res, api_engine = translate_via_libre(text, src, tgt)
    if api_res:
        return api_res, api_engine

    # 2) offline
    offline_res, offline_engine = translate_offline_opus(text, src, tgt)
    if offline_res:
        return offline_res, offline_engine

    # 3) fallback final (nenhuma tradução disponível)
    return text, "none:translation_failed"

# -------- EXEMPLOS DE USO (executar para testar) --------
print("\n=== TESTES DE TRADUÇÃO ===")
tests = [
    ("Olá, tudo bem com você?", "pt", "en"),
    ("How are you today?", "en", "pt"),
    ("Bom dia! Vamos estudar redes?", "pt", "en"),
    ("Buenos días, ¿cómo estás?", "es", "pt"),  # es->pt (OPUS disponível)
    ("This is a test of hybrid translator.", "en", "es")  # en->es available
]

for txt, s, t in tests:
    print("\n---")
    print("Orig:", txt)
    translated, engine = traduzir(txt, s, t)
    print("Trans:", translated)
    print("Engine:", engine)

# -------- INTERATIVO (opcional) --------
print("\nVocê pode chamar traduzir(texto, 'pt', 'en') diretamente para testar outros textos.")
# exemplo interativo:
traduzir("Escreva aqui o texto", "pt", "en")


Usando device: cpu

=== TESTES DE TRADUÇÃO ===

---
Orig: Olá, tudo bem com você?
🔄 Baixando e carregando modelo OPUS-MT: Helsinki-NLP/opus-mt-pt-en (pode demorar alguns segundos)...
Trans: Olá, tudo bem com você?
Engine: none:translation_failed

---
Orig: How are you today?
🔄 Baixando e carregando modelo OPUS-MT: Helsinki-NLP/opus-mt-en-pt (pode demorar alguns segundos)...
Trans: How are you today?
Engine: none:translation_failed

---
Orig: Bom dia! Vamos estudar redes?
🔄 Baixando e carregando modelo OPUS-MT: Helsinki-NLP/opus-mt-pt-en (pode demorar alguns segundos)...
Trans: Bom dia! Vamos estudar redes?
Engine: none:translation_failed

---
Orig: Buenos días, ¿cómo estás?
🔄 Baixando e carregando modelo OPUS-MT: Helsinki-NLP/opus-mt-es-pt (pode demorar alguns segundos)...
Trans: Buenos días, ¿cómo estás?
Engine: none:translation_failed

---
Orig: This is a test of hybrid translator.
🔄 Baixando e carregando modelo OPUS-MT: Helsinki-NLP/opus-mt-en-es (pode demorar alguns segundos)...
✅ 

('Escreva aqui o texto', 'none:translation_failed')

In [16]:
# ===============================================================
# ETAPA 2 – TRADUTOR COM GOOGLE TRANSLATE (googletrans)
# Versão interativa: usuário digita texto + escolhe idioma
# ===============================================================

# Instalar biblioteca (versão correta que funciona)
!pip install googletrans==4.0.0-rc1 --quiet

from googletrans import Translator

# Criar instância do tradutor
translator = Translator()

def traduzir(texto, origem, destino):
    """
    Realiza tradução usando o Google Translate.
    texto: string
    origem: código ISO (ex: 'pt', 'en', 'es')
    destino: código ISO
    """
    try:
        resultado = translator.translate(texto, src=origem, dest=destino)
        return resultado.text
    except Exception as e:
        return f"Erro na tradução: {e}"

# ---------------------------------------------------------------
# ENTRADA DO USUÁRIO (interativa no Colab)
# ---------------------------------------------------------------

print("=== TRADUTOR INTERATIVO (GOOGLE TRANSLATE) ===")
print("Digite o texto em português e escolha o idioma de destino.\n")

texto_usuario = input("Digite o texto para traduzir (em português): ")

print("\nIdiomas mais usados (códigos ISO):")
print("en = inglês")
print("es = espanhol")
print("fr = francês")
print("it = italiano")
print("de = alemão")
print("ja = japonês")
print("ko = coreano")
print("zh-cn = chinês simplificado")
print("ru = russo")
print("ar = árabe")

lingua_destino = input("\nDigite o código da língua desejada (ex: en, es, fr): ").lower().strip()

# ---------------------------------------------------------------
# EXECUTAR TRADUÇÃO
# ---------------------------------------------------------------
traducao_final = traduzir(texto_usuario, "pt", lingua_destino)

print("\n===== RESULTADO =====")
print("Texto original:", texto_usuario)
print(f"Tradução para {lingua_destino}:", traducao_final)
print("=====================")


=== TRADUTOR INTERATIVO (GOOGLE TRANSLATE) ===
Digite o texto em português e escolha o idioma de destino.

Digite o texto para traduzir (em português): computação em nuvem

Idiomas mais usados (códigos ISO):
en = inglês
es = espanhol
fr = francês
it = italiano
de = alemão
ja = japonês
ko = coreano
zh-cn = chinês simplificado
ru = russo
ar = árabe

Digite o código da língua desejada (ex: en, es, fr): ko

===== RESULTADO =====
Texto original: computação em nuvem
Tradução para ko: 클라우드 컴퓨팅
